In [11]:
# echo-server.py
import selectors
import socket
from _thread import *
import threading

LAN_IP = "127.0.0.1"  # Standard loopback interface address (localhost)
PORT = 9999  # Port to listen on (non-privileged ports are > 1023)

In [12]:
def parse_data(data):
    msg_type = None
    event = None
    sender_id = None
    relay = 0
    
    return msg_type


def thread(Client, Pserver):
    while True:
        # data received from client
        data = Pserver.recv(1024)
        if data:
            Client.send(data)
        elif not data:
            print_lock.release()
            break

#for tcp connections... unused
class Client:
    def __init__(self, connfd):
        self.connfd = connfd
    
    def send(self, data):
        try:
            self.connfd.send(data)
        except Exception as e:
            print(e)


class Vehicle:
    def __init__(self, LAN_IP="127.0.0.1", PORT=9999):
        self.location = 0 # 1 dimensional x coordinate
        self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.msg = ""
        self.buffer = ""
        self.prox_table = []
        self.velocity = 1 # m/s
        self.LAN_IP = LAN_IP
        self.PORT = PORT
        # --------- insert try statement for socket w broadcast -------- #
    
    def send_event_msg(self, prox_table):
        tmp_table = self.proximity_table
        try:
            self.s.sendall(request)
            print("request sent")
        except Exception as e:
            print(e)
        
        # read acks as they come in for 2s and then 
        
        # resend broadcast notification w updated 
        
        return 1
    
    def update_location(self, time):
        self.location = self.location + self.velocity * time
        return self.location
    
    
    def recieve(self):
        try:
            self.recieve_data = self.s.recv(4096)
            if self.recieve_data:
                print("recieved reply")
                return self.recieve_data
            elif not self.recieve_data:
                return None
        except Exception as e:
            print(e)
            
        def send_msg(self, msg, LAN_IP, PORT):
            sock.sendto(MESSAGE, (LAN_IP, PORT))

In [13]:
MESSAGE = b"Hello, World!"

In [14]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP

In [16]:
sock.sendto(MESSAGE, (LAN_IP, PORT))


13